# Convolutional Neural Network

### Importing libraries

In [49]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np

In [50]:
dir ="dataset"

## Part 1 - Data Preprocessing

In [51]:
train_datagen = ImageDataGenerator( rescale = 1./255,
                                    rotation_range = 30,
                                    shear_range = 0.2,
                                    zoom_range = 0.2,
                                    width_shift_range=0.05,
                                    height_shift_range=0.05,
                                    horizontal_flip = True,
                                    fill_mode = 'nearest',
                                    validation_split = 0.2)
training_set = train_datagen.flow_from_directory(dir,
                                                 target_size = (256, 256),
                                                 batch_size = 32,
                                                  subset= "training",
                                                class_mode = "categorical",
                                                color_mode = "rgb"
                                                 )


test_datagen = ImageDataGenerator(rescale = 1./255,
                                  validation_split=0.2
                                 )
test_set = test_datagen.flow_from_directory(dir,
                                            target_size = (256, 256),
                                            batch_size = 32,
                                             subset= "validation",
                                            class_mode = "categorical",
                                            color_mode = "rgb"
                                            )


Found 10019 images belonging to 10 classes.
Found 2502 images belonging to 10 classes.


## Part 2 - Building the CNN

In [52]:
cnn = tf.keras.models.Sequential()

cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[256, 256, 3]))

cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))

cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))

cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

cnn.add(tf.keras.layers.Flatten())

cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

cnn.add(tf.keras.layers.Dropout(0.2))

cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

cnn.add(tf.keras.layers.Dense(units=10, activation='softmax'))

cnn.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

cnn.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 127, 127, 32)     0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 125, 125, 32)      9248      
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 62, 62, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 60, 60, 32)        9248      
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 30, 30, 32)      

## Part 3 - Training the CNN

In [53]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 5)
cnn.save("model.h5")

Epoch 1/5
314/314 [==============================] - 243s 771ms/step - loss: 1.4326 - accuracy: 0.4996 - val_loss: 1.2195 - val_accuracy: 0.6223
Epoch 2/5
314/314 [==============================] - 240s 764ms/step - loss: 0.8491 - accuracy: 0.6994 - val_loss: 1.0398 - val_accuracy: 0.6875
Epoch 3/5
314/314 [==============================] - 246s 783ms/step - loss: 0.6754 - accuracy: 0.7659 - val_loss: 0.8480 - val_accuracy: 0.7454
Epoch 4/5
314/314 [==============================] - 253s 806ms/step - loss: 0.5734 - accuracy: 0.8021 - val_loss: 0.7053 - val_accuracy: 0.7838
Epoch 5/5
314/314 [==============================] - 232s 737ms/step - loss: 0.4928 - accuracy: 0.8214 - val_loss: 0.7142 - val_accuracy: 0.7654


## Part 4 - Making a single prediction

In [54]:

model=load_model('model.h5')


# Mention name of the disease into list.
Classes = ["Apple___Apple_scab","Apple___Black_rot","Apple___Cedar_apple_rust","Apple___healthy","Tomato___Bacterial_spot","Tomato___Early_blight",
"Tomato___healthy","Tomato___Late_blight","Tomato___Leaf_Mold","Tomato___Septoria_leaf_spot"]

def prepare(img_path):
    img = image.load_img(img_path, target_size=(256, 256))
    x = image.img_to_array(img)
    x = x/255
    return np.expand_dims(x, axis=0)
    

In [57]:
result = model.predict([prepare('dataset/Apple___Black_rot/00e909aa-e3ae-4558-9961-336bb0f35db3___JR_FrgE.S 8593.JPG')])
result = np.where(result[0]== np.amax(result[0]))
print(Classes[int(result[0])])


result = model.predict([prepare('dataset/Apple___healthy/0e3c0777-c214-4749-b8ae-4985274d2f9e___RS_HL 7984.JPG')])
result = np.where(result[0]== np.amax(result[0]))
print(Classes[int(result[0])])


result = model.predict([prepare('dataset/Apple___Cedar_apple_rust/809c7104-6f46-4117-8c8c-6d5491b2178d___FREC_C.Rust 3839.JPG')])
result = np.where(result[0]== np.amax(result[0]))
print(Classes[int(result[0])])


result = model.predict([prepare('dataset/Tomato___Leaf_Mold/0e5b669e-8534-45ab-a3c0-63134e685069___Crnl_L.Mold 7144.JPG')])
result = np.where(result[0]== np.amax(result[0]))
print(Classes[int(result[0])])


Apple___Black_rot
Apple___healthy
Apple___Cedar_apple_rust
Tomato___Late_blight
